# See the diff of (h-bond or non-h-bond) b/w strong and week attention by each head

- hbond combination / Attention strong combination.
- hbond combination / all combination.

In [1]:
%config IPCompleter.use_jedi=False

In [2]:
import os, sys
import subprocess
import pickle
from tqdm import tqdm
import pandas as pd
import numpy as np
from collections import defaultdict
from scipy.stats import ttest_ind_from_stats
from matplotlib import pyplot as plt

sys.path.append('../streamlit/')
from streamlit_utils import STD_THH, get_attention_and_hhb_relationship
from streamlit_utils import DICT_PDBID_2_CHAINNAMES, get_mapping_2_4digits
from streamlit_utils import DICT_PDBID_2_RESIDUES

from hhbond_stats_byhead import add_4digits

flatten = lambda xlist: [x for xx in xlist for x in xx]
pickleload = lambda p: pickle.load(open(p,"rb"))



loading....


  0%|                               | 0/4 [00:00<?, ?it/s]

ending... loading....


100%|███████████████████████| 4/4 [00:02<00:00,  1.39it/s]
25it [00:00, 58.76it/s]

2OL3 has only delta-chain and beta-chain


58it [00:02, 13.22it/s]

4QRR has only delta-chain and beta-chain


66it [00:02, 19.48it/s]

5JHD has only delta-chain and beta-chain
5JZI has only delta-chain and beta-chain


79it [00:02, 27.16it/s]
/Users/kyoheikoyama/.pyenv/versions/miniconda3-latest/envs/tcrpred/lib/python3.8/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [3]:
pd.options.display.float_format = '{:.5g}'.format

In [4]:
%%time

print('STD_THH =', STD_THH)

HEAD_COUNT = 4

# New
distance_path = './residue_distances.parquet'

if os.path.exists(distance_path):
    df_distance = pd.read_parquet(distance_path)
else:
    DATETIME = '20220211_191954'
    DICT_PDBID_2_MELTDIST = pickleload(
        f"/Users/kyoheikoyama/workspace/tcrpred/analysis/DICT_PDB_Result/{DATETIME}_DICT_PDBID_2_MELTDIST_allatoms.pickle"
    )
    df_distance = pd.concat([df.assign(pdbid=k) for k,df in DICT_PDBID_2_MELTDIST.items()])
    df_distance.to_parquet(distance_path)


STD_THH = 4.5
CPU times: user 60.9 ms, sys: 21.3 ms, total: 82.1 ms
Wall time: 82.7 ms


In [5]:
NEEDED_PDBIDS = ["2VLK", "5WKF", "3PQY", "4MJI", "4P2Q", "2YPL", "1J8H", "4P2R", "5MEN", "3MV8", "4OZF", "3VXR", "3VXS", "4OZG", "5TEZ", "2J8U", "6Q3S", "4JRX", "3VXU", "1U3H", "4JRY", "4Z7V", "4JFE", "4JFD", "3QIU", "2Z31", "2BNR", "3MBE", "4OZH", "2NX5", "5NHT", "4QOK", "5D2L", "1D9K", "4P2O", "5WKH", "6EQB", "2VLR", "6EQA"]
print(len(NEEDED_PDBIDS))


39


# Only more than x sigma and PosPred Attention 

In [6]:
givenPEP_distributedTCR_by_head_large, givenTCR_distributedPEP_by_head_large =\
get_attention_and_hhb_relationship(strong_atten=True)

df1_strong_atten = pd.concat(flatten([givenPEP_distributedTCR_by_head_large[hi] for hi in range(4)])) 
df2_strong_atten = pd.concat(flatten([givenTCR_distributedPEP_by_head_large[hi] for hi in range(4)]))

givenPEP_distributedTCR_by_head, givenTCR_distributedPEP_by_head =\
get_attention_and_hhb_relationship(strong_atten=False)

df1_normal_atten = pd.concat(flatten([givenPEP_distributedTCR_by_head[hi] for hi in range(4)])) 
df2_normal_atten = pd.concat(flatten([givenTCR_distributedPEP_by_head[hi] for hi in range(4)]))


100%|███████████████████████| 4/4 [00:03<00:00,  1.31it/s]


# Add minimum distance

In [7]:
dist_tcr_side = pd.merge(df_distance[['pdbid','tcr', 'value']].groupby(['pdbid','tcr'], as_index=False).min(), 
         df_distance, 
         on=['pdbid','tcr', 'value'],
        ).rename(columns={'value':'distance_value', 'peptide':'peptide_min'})

dist_peptide_side = pd.merge(df_distance[['pdbid','peptide', 'value']].groupby(['pdbid','peptide'], as_index=False).min(), 
         df_distance, 
         on=['pdbid','peptide', 'value'],
        ).rename(columns={'value':'distance_value', 'tcr':'tcr_min'})


df1_strong_atten = pd.merge(df1_strong_atten, dist_tcr_side, on=['pdbid','tcr'], how='left')
df1_normal_atten = pd.merge(df1_normal_atten, dist_tcr_side, on=['pdbid','tcr'], how='left' )
df2_strong_atten = pd.merge(df2_strong_atten, dist_peptide_side, on=['pdbid','peptide'], how='left' )
df2_normal_atten = pd.merge(df2_normal_atten, dist_peptide_side, on=['pdbid','peptide'], how='left' )

df1_strong_atten = df1_strong_atten.query('pdbid in @NEEDED_PDBIDS')
df1_normal_atten = df1_normal_atten.query('pdbid in @NEEDED_PDBIDS')
df2_strong_atten = df2_strong_atten.query('pdbid in @NEEDED_PDBIDS')
df2_normal_atten = df2_normal_atten.query('pdbid in @NEEDED_PDBIDS')

df_tcr_side = pd.concat([df1_strong_atten.assign(is_large_atten=True), df1_normal_atten.assign(is_large_atten=False), ])
df_tcr_side = df_tcr_side.assign(is_tcr_side=True)
df_pep_side = pd.concat([df2_strong_atten.assign(is_large_atten=True), df2_normal_atten.assign(is_large_atten=False), ])
df_pep_side = df_pep_side.assign(is_tcr_side=False)
df_pep_side

,peptide,tcr,value_x,value_y,attention_original,attention_log10,pdbid,head,distance_value,tcr_min,is_large_atten,is_tcr_side
0,I_7,A_14,13.643,0.54368,0.54368,-0.26466,2YPL,0,3.2118,Y_6,True,False
1,V_3,S_17,21.323,0.44758,0.44758,-0.34913,4P2R,0,3.2836,N_6,True,False
2,L_5,A_21,3.5747,0.5087,0.5087,-0.29354,4OZF,0,3.5747,A_21,True,False
3,R_8,C_14,20.871,0.68302,0.68302,-0.16557,1U3H,0,2.61,Y_23,True,False
4,G_7,Y_23,10.17,0.48898,0.48898,-0.31071,4JRY,0,2.9548,R_18,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
55257,V_8,T_21,13.218,0.096361,0.096361,-1.0161,6EQA,3,7.7018,T_17,False,False
55258,V_8,T_9,24.827,0.025839,0.025839,-1.5877,6EQA,3,7.7018,T_17,False,False
55259,V_8,V_2,23.696,0.040502,0.040502,-1.3925,6EQA,3,7.7018,T_17,False,False
55260,V_8,V_4,21.409,0.023304,0.023304,-1.6326,6EQA,3,7.7018,T_17,False,False


# Compare Properties

## Take the min by each head

In [8]:
DICT_bond_tuple_fromto = {}

from streamlit_utils import read_hhb_file, get_mapping_2_4digits, DICT_PDBID_2_CHAINNAMES

def read_hhb_text_and_find_chain(text, chainsletters):
    ress_atoms = text[:45].replace('  ',' ').replace('  ',' ').replace('  ',' ').replace('  ',' ').split(' ')
    donor_res = ress_atoms[0]
    acceptor_res = ress_atoms[2]
    if donor_res.split('/')[1][:1] in chainsletters or \
            acceptor_res.split('/')[1][:1] in chainsletters:
        return text


def create_bond_info_by_pdb(pdbid):
    chains = DICT_PDBID_2_CHAINNAMES[pdbid.upper()]
    chainsletters = ''.join(chains)

    dfhhb = read_hhb_file(pdbid)
    donors_n_acceptors = [read_hhb_text_and_find_chain(row.item(), chainsletters) for i, row in dfhhb.iterrows()]
    donors_n_acceptors = [a for a in donors_n_acceptors if a is not None]
    
    bond_tuple_fromto = []
    for da in donors_n_acceptors:
        a = da.replace('  ',' ').replace('  ',' ').replace('  ',' ').replace('  ',' ').split(' ')
        from_d = a[0].split('/')[1].split('-')[0][:5]
        to_d = a[2].split('/')[1].split('-')[0][:5]
        bond_tuple_fromto.append((from_d, to_d))
        
    DICT_bond_tuple_fromto[pdbid] = bond_tuple_fromto
    anyhbond_related_4digits = list(set([tup[0] for tup in bond_tuple_fromto] + [tup[1] for tup in bond_tuple_fromto]))

    CDRname_2_4digits, Peptide_2_4digits, bond_list = get_mapping_2_4digits(donors_n_acceptors, pdbid)
    dfc = pd.DataFrame(CDRname_2_4digits.items(), columns=['residue', 'digit4']).assign(**{'is_tcr':True, 'pdbid':pdbid})
    dfp = pd.DataFrame(Peptide_2_4digits.items(), columns=['residue', 'digit4']).assign(**{'is_tcr':False, 'pdbid':pdbid})
    df = pd.concat([dfc,dfp])
    df['has_bond'] = df['digit4'].isin(anyhbond_related_4digits)
    return df
    

In [9]:

def generate_alpha_beta_pep():
    alis, blis, clis = [],[],[]
    for pdbid in NEEDED_PDBIDS:
        residues = DICT_PDBID_2_RESIDUES[pdbid]
        df_alpha = pd.DataFrame([residues[0]]).T.assign(pdbid=pdbid)
        df_alpha['digit4'] = df_alpha[0].apply(add_4digits)
        df_beta = pd.DataFrame([residues[1]]).T.assign(pdbid=pdbid)
        df_beta['digit4'] = df_beta[0].apply(add_4digits)
        df_pep = pd.DataFrame([residues[2]]).T.assign(pdbid=pdbid)
        df_pep['digit4'] = df_pep[0].apply(add_4digits)
        alis.append(df_alpha)
        blis.append(df_beta)
        clis.append(df_pep)
    return pd.concat(alis),pd.concat(blis),pd.concat(clis)

df_alpha, df_beta, df_pep = generate_alpha_beta_pep()

In [10]:
df_bondinfo = pd.concat([create_bond_info_by_pdb(p) for p in NEEDED_PDBIDS])
df_bond_tuple_fromto = pd.DataFrame(DICT_bond_tuple_fromto.items(), columns=['pdbid', 'bonds'])

df_bond_relations = []
for i, row in df_bond_tuple_fromto.iterrows():
    list_of_tuple = row['bonds']
    pdbid = row['pdbid']
    bond_dict = defaultdict(list)
    for tup in list_of_tuple:
        bond_dict[tup[0]].append(tup[1])
        bond_dict[tup[1]].append(tup[0])
    t = pd.DataFrame(bond_dict.items(), columns=['from_res', 'to_res_list']).assign(pdbid=pdbid)
    df_bond_relations.append(t)
df_bond_relations = pd.concat(df_bond_relations)


# Create info

In [11]:
df_tcr_side.head(3) # info of tcr-side attention given a pdbid and given a head

,peptide,tcr,value_x,value_y,attention_original,attention_log10,pdbid,head,distance_value,peptide_min,is_large_atten,is_tcr_side
0,F_4,G_2,10.348,0.69898,0.69898,-0.15553,2VLK,0,10.348,F_4,True,True
1,L_8,G_2,20.52,0.65179,0.65179,-0.18589,2VLK,0,10.348,F_4,True,True
2,T_7,G_2,17.342,0.72168,0.72168,-0.14166,2VLK,0,10.348,F_4,True,True


In [12]:
df_bondinfo = pd.merge(df_bondinfo, df_bond_relations, 
                       left_on=['pdbid', 'digit4'], right_on=['pdbid', 'from_res'], 
                       how='left')

In [13]:
df_bondinfo['to_res_list'] = df_bondinfo.to_res_list.apply(lambda xlis: [] if not isinstance(xlis, list) else xlis)

In [14]:
# is_connecting_tcr
df_bondinfo['is_connecting_to_cdr'] = False
for i, row in tqdm(df_bondinfo.iterrows()):
    if isinstance(row.to_res_list, list):
        p = row.pdbid
        cdr_list = df_bondinfo.query('is_tcr==True and pdbid==@p')['digit4'].tolist()
        is_any_res_in_cdr = any([(r in cdr_list) for r in row.to_res_list])
        df_bondinfo.loc[[i], 'is_connecting_to_cdr'] = is_any_res_in_cdr
    
del cdr_list, is_any_res_in_cdr

1474it [00:04, 311.20it/s]


In [15]:
df_bondinfo['is_connecting_to_pep'] = False
for i, row in tqdm(df_bondinfo.iterrows()):
    if isinstance(row.to_res_list, list):
        p = row.pdbid
        pep_list = df_bondinfo.query('is_tcr==False and pdbid==@p')['digit4'].tolist()
        is_any_res_in_pep = any([(r in pep_list) for r in row.to_res_list])
        df_bondinfo.loc[[i], 'is_connecting_to_pep']  = is_any_res_in_pep
del pep_list, is_any_res_in_pep

1474it [00:05, 276.84it/s]


In [16]:
df_bondinfo['is_connecting_to_tcr'] = False
for i, row in tqdm(df_bondinfo.iterrows()):
    if isinstance(row.to_res_list, list):
        p = row.pdbid
        tcr_list =\
            [t for t in df_alpha.query('pdbid==@p')['digit4'].tolist()] +\
            [t for t in df_beta.query('pdbid==@p')['digit4'].tolist()]
        is_tcr_list = any([(r in tcr_list) for r in row.to_res_list])
        df_bondinfo.loc[[i], 'is_connecting_to_tcr']  = is_tcr_list
        
del tcr_list, is_tcr_list

1474it [00:05, 273.53it/s]


In [17]:
df_bondinfo['is_connecting_to_notCDR_tcr'] = False
for i, row in tqdm(df_bondinfo.iterrows()):
    if isinstance(row.to_res_list, list):
        p = row.pdbid
        cdr_list = df_bondinfo.query('is_tcr==True and pdbid==@p')['digit4'].tolist()
        notcdr_tcr_list =\
            [t for t in df_alpha.query('pdbid==@p')['digit4'].tolist() if t not in cdr_list] +\
            [t for t in df_beta.query('pdbid==@p')['digit4'].tolist() if t not in cdr_list]
        is_notcdr_tcr_list = any([(r in notcdr_tcr_list) for r in row.to_res_list])
        df_bondinfo.loc[[i], 'is_connecting_to_notCDR_tcr']  = is_notcdr_tcr_list
del notcdr_tcr_list, is_notcdr_tcr_list


1474it [00:10, 145.02it/s]


In [18]:
df_bondinfo

,residue,digit4,is_tcr,pdbid,has_bond,from_res,to_res_list,is_connecting_to_cdr,is_connecting_to_pep,is_connecting_to_tcr,is_connecting_to_notCDR_tcr
0,C_0,D0090,True,2VLK,True,D0090,"[D0007, D0102]",False,False,True,True
1,A_1,D0091,True,2VLK,True,D0091,"[D0034, D0034]",False,False,True,True
2,G_2,D0092,True,2VLK,True,D0092,"[D0100, D0100]",True,False,True,False
3,A_3,D0093,True,2VLK,True,D0093,"[D0003, D0032, D2029]",False,False,True,True
4,G_4,D0094,True,2VLK,True,D0094,"[D0098, D0097, E0100]",True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...
1469,G_4,C0006,False,6EQA,True,C0006,"[A0155, E0098, E0098]",True,False,True,False
1470,I_5,C0007,False,6EQA,True,C0007,"[E0098, E0098, E0098, E0098]",True,False,True,False
1471,L_6,C0008,False,6EQA,False,NaN,[],False,False,False,False
1472,T_7,C0009,False,6EQA,True,C0009,"[A0146, A0147]",False,False,False,False


In [19]:
df_bondinfo['is_connecting_to_ownchain_tcr'] = False
for i, row in tqdm(df_bondinfo.iterrows()):
    if row.is_tcr==False:
        continue
    is_connecting_to_ownchain_tcr = False
    p = row.pdbid
    alpha_digits_list = df_alpha.query('pdbid==@p')['digit4'].tolist()
    beta_digits_list = df_beta.query('pdbid==@p')['digit4'].tolist()
    is_in_alpha = row.digit4 in alpha_digits_list
    is_in_beta = row.digit4 in beta_digits_list
    assert (is_in_alpha+is_in_beta) > 0
    if is_in_alpha:
        is_connecting_to_ownchain_tcr = any([(r in alpha_digits_list) for r in row.to_res_list])
    if is_in_beta:
        is_connecting_to_ownchain_tcr = any([(r in beta_digits_list) for r in row.to_res_list])

    df_bondinfo.loc[[i], 'is_connecting_to_ownchain_tcr']  = is_connecting_to_ownchain_tcr
    
# del alpha_digits_list, beta_digits_list


1474it [00:03, 369.15it/s]


In [20]:
df_bondinfo['is_connecting_to_ownchain_cdr'] = False
for i, row in tqdm(df_bondinfo.iterrows()):
    if row.is_tcr==False:
        continue
    is_connecting_to_ownchain_cdr = False
    p = row.pdbid
    tcr_list = df_bondinfo.query('is_tcr==True and pdbid==@p')['digit4'].tolist()
    alpha_digits_list = df_alpha.query('pdbid==@p and (digit4 in @tcr_list)')['digit4'].tolist()
    beta_digits_list = df_beta.query('pdbid==@p and (digit4 in @tcr_list)')['digit4'].tolist()
    is_in_alpha = row.digit4 in alpha_digits_list
    is_in_beta = row.digit4 in beta_digits_list
    assert (is_in_alpha+is_in_beta) > 0
    if is_in_alpha:
        is_connecting_to_ownchain_cdr = any([(r in alpha_digits_list) for r in row.to_res_list])
    if is_in_beta:
        is_connecting_to_ownchain_cdr = any([(r in beta_digits_list) for r in row.to_res_list])

    df_bondinfo.loc[[i], 'is_connecting_to_ownchain_cdr']  = is_connecting_to_ownchain_cdr
    
# del alpha_digits_list, beta_digits_list


1474it [00:09, 148.74it/s]


In [21]:
df_bondinfo['is_connecting_to_opposite_chain_tcr'] = False
for i, row in tqdm(df_bondinfo.iterrows()):
    if row.is_tcr==False:
        continue
    is_opposite = False
    p = row.pdbid
    # tcr_list = df_bondinfo.query('is_tcr==True and pdbid==@p')['digit4'].tolist()
    alpha_digits_list = df_alpha.query('pdbid==@p ')['digit4'].tolist()
    beta_digits_list = df_beta.query('pdbid==@p ')['digit4'].tolist()
    is_in_alpha = row.digit4 in alpha_digits_list
    is_in_beta = row.digit4 in beta_digits_list
    assert (is_in_alpha+is_in_beta) > 0
    if is_in_alpha:
        is_opposite = any([(r in beta_digits_list) for r in row.to_res_list])
    if is_in_beta:
        is_opposite = any([(r in alpha_digits_list) for r in row.to_res_list])

    df_bondinfo.loc[[i], 'is_connecting_to_opposite_chain_tcr']  = is_opposite
    
# del alpha_digits_list, beta_digits_list


1474it [00:04, 323.30it/s]


In [22]:
df_bondinfo['is_connecting_to_opposite_chain_cdr'] = False
for i, row in tqdm(df_bondinfo.iterrows()):
    if row.is_tcr==False:
        continue
    is_opposite = False
    p = row.pdbid
    tcr_list = df_bondinfo.query('is_tcr==True and pdbid==@p')['digit4'].tolist()
    alpha_digits_list = df_alpha.query('pdbid==@p and (digit4 in @tcr_list)')['digit4'].tolist()
    beta_digits_list = df_beta.query('pdbid==@p and (digit4 in @tcr_list)')['digit4'].tolist()
    is_in_alpha = row.digit4 in alpha_digits_list
    is_in_beta = row.digit4 in beta_digits_list
    assert (is_in_alpha+is_in_beta) > 0
    if is_in_alpha:
        is_opposite = any([(r in beta_digits_list) for r in row.to_res_list])
    if is_in_beta:
        is_opposite = any([(r in alpha_digits_list) for r in row.to_res_list])

    df_bondinfo.loc[[i], 'is_connecting_to_opposite_chain_cdr']  = is_opposite
    
# del alpha_digits_list, beta_digits_list


1474it [00:11, 132.92it/s]


In [23]:
df_bondinfo['num_bonds'] = df_bondinfo['to_res_list'].apply(len)

In [24]:
df_bondinfo['digit4_is_in_edge'] = False
for i, row in tqdm(df_bondinfo.iterrows()):
    p = row.pdbid
    EDGENUM = 4
    residue_list = df_bondinfo.query('is_tcr==True and pdbid==@p')['residue'].tolist()
    pos_all = pd.Series(residue_list).str.split('_').apply(lambda x: x[1]).values.astype(int)
    colon_pos = [i for i in range(len(pos_all)) if i not in pos_all][0]
    beginnings_endings_cdrs = list(range(EDGENUM)) + pos_all[-EDGENUM:].tolist() + \
                                          [colon_pos-i-1 for i in range(EDGENUM)] + \
                                          [colon_pos+i+1 for i in range(EDGENUM)]
    
    df_bondinfo.loc[[i], 'digit4_is_in_edge']  = int(row.residue.split('_')[1]) in beginnings_endings_cdrs


1474it [00:06, 222.00it/s]


In [25]:
df_bondinfo.head(3)

,residue,digit4,is_tcr,pdbid,has_bond,from_res,to_res_list,is_connecting_to_cdr,is_connecting_to_pep,is_connecting_to_tcr,is_connecting_to_notCDR_tcr,is_connecting_to_ownchain_tcr,is_connecting_to_ownchain_cdr,is_connecting_to_opposite_chain_tcr,is_connecting_to_opposite_chain_cdr,num_bonds,digit4_is_in_edge
0,C_0,D0090,True,2VLK,True,D0090,"[D0007, D0102]",False,False,True,True,True,False,False,False,2,True
1,A_1,D0091,True,2VLK,True,D0091,"[D0034, D0034]",False,False,True,True,True,False,False,False,2,True
2,G_2,D0092,True,2VLK,True,D0092,"[D0100, D0100]",True,False,True,False,True,True,False,False,2,True


In [26]:
df_bondinfo['is_connecting_to_cdr'].value_counts(),\
df_bondinfo['is_connecting_to_pep'].value_counts(),\
df_bondinfo['is_connecting_to_notCDR_tcr'].value_counts()

(False    957
 True     517
 Name: is_connecting_to_cdr, dtype: int64,
 False    1362
 True      112
 Name: is_connecting_to_pep, dtype: int64,
 False    972
 True     502
 Name: is_connecting_to_notCDR_tcr, dtype: int64)

# Merge

In [27]:
df_tcr_byhead = pd.concat([
    df_tcr_side.groupby(['pdbid', 'tcr', 'head']).max()[['is_large_atten',]],
    df_tcr_side.groupby(['pdbid', 'tcr', 'head']).min()[['distance_value','peptide_min',]],
                          ], axis=1).reset_index()
df_tcr_byhead = pd.merge(df_tcr_byhead,
                         df_bondinfo.query('is_tcr==True'), 
                         how='left', 
                         left_on=['pdbid', 'tcr'], 
                         right_on=['pdbid', 'residue'], )


df_tcr_allhead = pd.concat([
    df_tcr_side.groupby(['pdbid', 'tcr', ]).max()[['is_large_atten',]],
    df_tcr_side.groupby(['pdbid', 'tcr', ]).min()[['distance_value','peptide_min',]],
                          ], axis=1).reset_index()
df_tcr_allhead = pd.merge(df_tcr_allhead,
                          df_bondinfo.query('is_tcr==True'), how='left', 
             left_on=['pdbid', 'tcr'], 
             right_on=['pdbid', 'residue'], )

# Contingency table and Fisher's test

In [28]:
from scipy import stats
prop_list = ['is_connecting_to_pep', 'is_connecting_to_cdr', 
             'is_connecting_to_ownchain_tcr','is_connecting_to_ownchain_cdr',
             'is_connecting_to_opposite_chain_tcr', 'is_connecting_to_opposite_chain_cdr',
             'is_connecting_to_tcr', 'is_connecting_to_notCDR_tcr',
             'digit4_is_in_edge', 'distance_value', 'num_bonds',
            ]

In [29]:
print([c for c in df_tcr_allhead.columns if 'is_' in c])

['is_large_atten', 'is_tcr', 'is_connecting_to_cdr', 'is_connecting_to_pep', 'is_connecting_to_tcr', 'is_connecting_to_notCDR_tcr', 'is_connecting_to_ownchain_tcr', 'is_connecting_to_ownchain_cdr', 'is_connecting_to_opposite_chain_tcr', 'is_connecting_to_opposite_chain_cdr', 'digit4_is_in_edge']


## All heads combined

In [30]:
for prop in prop_list:
    print()
    print(prop)
    pd.options.display.float_format = '{:.3g}'.format
    means = (df_tcr_allhead.groupby('is_large_atten').mean().sort_index(ascending=False)[prop]).values
    stds = (df_tcr_allhead.groupby('is_large_atten').std().sort_index(ascending=False)[prop]).values
    obs = df_tcr_allhead.groupby('is_large_atten').agg(len).sort_index(ascending=False).iloc[:,0].values
    ttest_result = stats.ttest_ind_from_stats(mean1=means[0], mean2=means[1], 
                               std1=stds[0], std2=stds[1],
                               nobs1=obs[0], nobs2=obs[1],
                              )
    print('  ', ttest_result)
    print(f"\t mean+std of large = {means[0]:.5f}+-{stds[0]:.5f}", f'n_obs =',obs[0])
    print(f"\t mean+std of small = {means[1]:.5f}+-{stds[1]:.5f}", f'n_obs =',obs[1] )



is_connecting_to_pep
   Ttest_indResult(statistic=0.2699325564961965, pvalue=0.7872659499351644)
	 mean+std of large = 0.08673+-0.28217 n_obs = 196
	 mean+std of small = 0.08086+-0.27278 n_obs = 841

is_connecting_to_cdr
   Ttest_indResult(statistic=1.5790499973653787, pvalue=0.11463014249105333)
	 mean+std of large = 0.47449+-0.50063 n_obs = 196
	 mean+std of small = 0.41260+-0.49260 n_obs = 841

is_connecting_to_ownchain_tcr
   Ttest_indResult(statistic=-1.5661220319083478, pvalue=0.11762572418661102)
	 mean+std of large = 0.59694+-0.49177 n_obs = 196
	 mean+std of small = 0.65636+-0.47520 n_obs = 841

is_connecting_to_ownchain_cdr
   Ttest_indResult(statistic=1.9947309029883005, pvalue=0.04633413789907231)
	 mean+std of large = 0.45408+-0.49916 n_obs = 196
	 mean+std of small = 0.37693+-0.48491 n_obs = 841

is_connecting_to_opposite_chain_tcr
   Ttest_indResult(statistic=0.5623966335509497, pvalue=0.5739675943345927)
	 mean+std of large = 0.16837+-0.37515 n_obs = 196
	 mean+std of 

## Fisher Contingency All heads

In [31]:
df_tcr_allhead.groupby('is_large_atten').agg(len).sort_index(ascending=False).iloc[:,0].values

array([196, 841])

In [32]:


for prop in prop_list:
    if prop=='distance_value' or prop=='num_bonds':
        print(prop)
        pd.options.display.float_format = '{:.3g}'.format
        means = (df_tcr_allhead.groupby('is_large_atten').mean().sort_index(ascending=False)[prop]).values
        stds = (df_tcr_allhead.groupby('is_large_atten').std().sort_index(ascending=False)[prop]).values
        obs = df_tcr_allhead.groupby('is_large_atten').agg(len).sort_index(ascending=False).iloc[:,0].values
        ttest_result = stats.ttest_ind_from_stats(mean1=means[0], mean2=means[1], 
                                   std1=stds[0], std2=stds[1],
                                   nobs1=obs[0], nobs2=obs[1],
                                  )
        print(prop, ttest_result)
        print(f"mean+std of large = {means[0]}+-{stds[0]}", )
        print(f"mean+std of small = {means[1]}+-{stds[1]}", )
        continue
    
    print()
    print('property', prop)
    df_margin = pd.crosstab(df_tcr_allhead.is_large_atten, df_tcr_allhead[prop], margins=True)
    display(df_margin)
    df = pd.crosstab(df_tcr_allhead.is_large_atten, df_tcr_allhead[prop], dropna=False)
    oddsr, p = stats.fisher_exact(table=df.to_numpy(), alternative='two-sided')
    
    print(f'oddsr={oddsr:.5f}, p={p:.5f}')
    if p<0.1:
        print(f'Significant! head=all, prop={prop}, p={p}')
    


property is_connecting_to_pep


is_connecting_to_pep,False,True,All
is_large_atten,,,
False,773,68,841
True,179,17,196
All,952,85,1037


oddsr=1.07961, p=0.77315

property is_connecting_to_cdr


is_connecting_to_cdr,False,True,All
is_large_atten,,,
False,494,347,841
True,103,93,196
All,597,440,1037


oddsr=1.28541, p=0.12732

property is_connecting_to_ownchain_tcr


...

oddsr=0.79904, p=0.21541

property is_connecting_to_notCDR_tcr


is_connecting_to_notCDR_tcr,False,True,All
is_large_atten,,,
False,448,393,841
True,137,59,196
All,585,452,1037


oddsr=0.49093, p=0.00002
Significant! head=all, prop=is_connecting_to_notCDR_tcr, p=2.1094816738816693e-05

property digit4_is_in_edge


digit4_is_in_edge,False,True,All
is_large_atten,,,
False,344,497,841
True,69,127,196
All,413,624,1037


oddsr=1.27396, p=0.14574
distance_value
distance_value Ttest_indResult(statistic=0.022381353779624455, pvalue=0.9821480681761573)
mean+std of large = 8.420896464464615+-4.098932307005889
mean+std of small = 8.412923731786885+-4.577395950757676
num_bonds
num_bonds Ttest_indResult(statistic=-0.6109985046172097, pvalue=0.5413347274034115)
mean+std of large = 2.020408163265306+-1.6043504434408704
mean+std of small = 2.0963139120095127+-1.5573170507574983


In [33]:
print(df_tcr_allhead.groupby('is_large_atten').mean()[['distance_value']], df_tcr_allhead.groupby('is_large_atten').std()[['distance_value']],)
print()
print(stats.ttest_ind(
    df_tcr_allhead.query('is_large_atten==True')['distance_value'],
    df_tcr_allhead.query('is_large_atten==False')['distance_value']
))

                distance_value
is_large_atten                
False                     8.41
True                      8.42                 distance_value
is_large_atten                
False                     4.58
True                       4.1

Ttest_indResult(statistic=0.022381353779624465, pvalue=0.9821480681761572)


## By head

In [34]:
for hhh in range(4):
    print('-'*30)
    print('head', hhh)
    for prop in prop_list:
        temp = df_tcr_byhead.query('head==@hhh')
        if prop in ['distance_value', 'num_bonds']:
            pd.options.display.float_format = '{:.3g}'.format
            means = (temp.groupby('is_large_atten').mean().sort_index(ascending=False)[['distance_value']].T).values[0]
            stds = (temp.groupby('is_large_atten').std().sort_index(ascending=False)[['distance_value']].T).values[0]
            print(f"mean+std of large = {means[0]}+-{stds[0]}", )
            print(f"mean+std of small = {means[1]}+-{stds[1]}", )
            continue
        
        print()
        print('property', prop)
        df_margin = pd.crosstab(temp.is_large_atten, temp[prop], margins=True)
        display(df_margin)
        df = pd.crosstab(temp.is_large_atten, temp[prop], margins=False)
        if True not in df.columns:df[True]=[0,0]
        oddsr, p = stats.fisher_exact(table=df.to_numpy(), alternative='two-sided')
        print(f'oddsr={oddsr:.5f}, p={p:.5f}')
        if p<0.1:
            print(f'Significant! head={hhh}, prop={prop}, p={p}')


------------------------------
head 0

property is_connecting_to_pep


is_connecting_to_pep,False,True,All
is_large_atten,,,
False,898,84,982
True,54,1,55
All,952,85,1037


oddsr=0.19797, p=0.07915
Significant! head=0, prop=is_connecting_to_pep, p=0.07915234402981997

property is_connecting_to_cdr


is_connecting_to_cdr,False,True,All
is_large_atten,,,
False,574,408,982
True,23,32,55
All,597,440,1037


oddsr=1.95737, p=0.01712
Significant! head=0, prop=is_connecting_to_cdr, p=0.01711954872466933

property is_connecting_to_ownchain_tcr


...

oddsr=0.77999, p=0.37932

property is_connecting_to_notCDR_tcr


is_connecting_to_notCDR_tcr,False,True,All
is_large_atten,,,
False,548,428,976
True,37,24,61
All,585,452,1037


oddsr=0.83051, p=0.50924

property digit4_is_in_edge


digit4_is_in_edge,False,True,All
is_large_atten,,,
False,396,580,976
True,17,44,61
All,413,624,1037


oddsr=1.76714, p=0.05859
Significant! head=3, prop=digit4_is_in_edge, p=0.058588614939087943
mean+std of large = 8.417055262893927+-3.9557175807899623
mean+std of small = 8.414266592655025+-4.522029909909692
mean+std of large = 8.417055262893927+-3.9557175807899623
mean+std of small = 8.414266592655025+-4.522029909909692


# Dependent Student t-test by PDB

## all-head

In [35]:
(df[True] / df.sum(axis=1)) .sort_index(ascending=False)

is_large_atten
True    0.721
False   0.594
dtype: float64

In [36]:
def return_property_by_islarge(temp, prop):
    if prop not in ['distance_value', 'num_bonds']:
        temp = temp.groupby(['pdbid', 'tcr', ]).max().reset_index()
        df = pd.crosstab(temp.is_large_atten, temp[prop], margins=False, )
        if True not in df.columns:
            df[True] = [0 for _ in range(len(df))]
        if (len(df.columns)!=2) or (len(df.index)!=2):
            return [np.nan, (df[True] / df.sum(axis=1)).loc[False]]
            
        df_margin = pd.crosstab(temp.is_large_atten, temp[prop], margins=True, normalize=True)
        oddsr, p = stats.fisher_exact(table=df.to_numpy(), alternative='two-sided')
        return (df[True] / df.sum(axis=1)).sort_index(ascending=False).values
    else:
        temp = pd.concat([
            temp.groupby(['tcr', ]).max()[['is_large_atten','digit4_is_in_edge']],
            temp.groupby(['tcr', ]).min()[['distance_value','peptide_min',]],
            temp.groupby(['tcr', ]).mean()[['num_bonds',]],
        ], axis=1).reset_index()

        temp_mean = temp.groupby('is_large_atten').mean()[prop]
        if len(temp_mean)==1:
            return  [np.nan, temp_mean.loc[False]]
        return [temp_mean.loc[True], temp_mean.loc[False]]
    

In [48]:
table_all = []
for prop in prop_list:
    vals = []
    vals2 = []
    print('-'*30)
    for ppp in df_tcr_byhead.pdbid.unique():     
        temp = df_tcr_byhead.query('pdbid==@ppp')
        vals2.append(return_property_by_islarge(temp, prop))

    vals = np.array(vals2)
    ttest_result = stats.ttest_rel(vals[:,0], vals[:,1],nan_policy='omit')
    print('prop', prop)
    means = np.array(vals).mean(axis=0).tolist()
    stds = np.array(vals).std(axis=0)
    if np.isnan(means[0]):
        assert False
    table_all.append(['Proportion ' + prop if 'is_' in prop else prop, 
                 f"{means[0]:.4f}+-{stds[0]:.4f}", 
                 f"{means[1]:.4f}+-{stds[1]:.4f}", 
                 ttest_result[1],
                 'all']
                )        
    print(f"mean+std of large = {means[0]}+-{stds[0]}", )
    print(f"mean+std of small = {means[1]}+-{stds[1]}", )
    print("std of (large, small) = ", )
    print('ttest_result', ttest_result)
    if ttest_result[1]<0.05:
        print('*** Significant!')
    else:
        print('#### not significant...')

------------------------------
prop is_connecting_to_pep
mean+std of large = 0.08623321123321123+-0.13676382161617467
mean+std of small = 0.08045637666655092+-0.06751997495415424
std of (large, small) = 
ttest_result Ttest_relResult(statistic=0.21916075951560385, pvalue=0.8276977642280666)
#### not significant...
------------------------------
prop is_connecting_to_cdr
mean+std of large = 0.4846459096459098+-0.22158980221033536
mean+std of small = 0.41026050207884385+-0.10400610724367566
std of (large, small) = 
ttest_result Ttest_relResult(statistic=2.0447748432126485, pvalue=0.047848936104267846)
*** Significant!
------------------------------
prop is_connecting_to_ownchain_tcr
mean+std of large = 0.6012515262515264+-0.19994056705998162
mean+std of small = 0.6560852212128397+-0.08802387215549679
std of (large, small) = 
ttest_result Ttest_relResult(statistic=-1.6032203266607086, pvalue=0.11716684195158339)
#### not significant...
------------------------------
prop is_connecting_to_o

In [51]:
table_all = pd.DataFrame(table_all, 
             columns=['property', 'Large Atten Mean. Mean(STD)', 'Small Atten. Mean(STD)', 'P Value', 'Head'])
table_all.loc[table_all['P Value'] < 0.10, ' '] = '*'
table_all.loc[table_all['P Value'] < 0.05, ' '] = '***'
table_all.loc[table_all['P Value'] >= 0.05, ' '] = ''
table_all[' '].fillna(' ')
table_all

,property,Large Atten Mean. Mean(STD),Small Atten. Mean(STD),P Value,Head,
0,Proportion is_connecting_to_pep,0.0862+-0.1368,0.0805+-0.0675,0.828,all,
1,Proportion is_connecting_to_cdr,0.4846+-0.2216,0.4103+-0.1040,0.0478,all,***
2,Proportion is_connecting_to_ownchain_tcr,0.6013+-0.1999,0.6561+-0.0880,0.117,all,
3,Proportion is_connecting_to_ownchain_cdr,0.4643+-0.2180,0.3752+-0.0922,0.0107,all,***
4,Proportion is_connecting_to_opposite_chain_tcr,0.1679+-0.1714,0.1497+-0.0793,0.562,all,
5,Proportion is_connecting_to_opposite_chain_cdr,0.0306+-0.0857,0.0672+-0.0743,0.0369,all,***
6,Proportion is_connecting_to_tcr,0.6845+-0.1650,0.7294+-0.0880,0.0987,all,
7,Proportion is_connecting_to_notCDR_tcr,0.2940+-0.1923,0.4672+-0.0846,3.88e-05,all,***
8,Proportion digit4_is_in_edge,0.6434+-0.2064,0.5928+-0.0570,0.218,all,
9,distance_value,8.4072+-2.2892,8.4122+-0.9592,0.988,all,


## By head

In [38]:
table = []
table_by_pdb = []
for hhh in range(4):
    print()
    print()
    print('head',hhh, '*'*30)
    df_by_prop = []
    for prop in prop_list:
        vals = []
        for ppp in df_tcr_byhead.pdbid.unique():     
            temp = df_tcr_byhead.query('pdbid==@ppp and head==@hhh')
            vals.append(return_property_by_islarge(temp, prop))
        
        vals = np.array(vals)
        
        df_minimum = pd.DataFrame(vals, index=df_tcr_byhead.pdbid.unique(), columns=[f'Large {prop}', f'Small {prop}'])
        
        ttest_result = stats.ttest_rel(vals[:,0], vals[:,1], nan_policy='omit')
        means = np.nanmean(vals, axis=0).tolist()
        stds = np.nanstd(vals, axis=0)
        if np.isnan(means[0]):
            assert False
        print()
        print('  prop', prop)
        print(f"\t mean+std of large = {means[0]}+-{stds[0]}", )
        print(f"\t mean+std of small = {means[1]}+-{stds[1]}", )
        # print("std of (large, small) = ", )
        print(ttest_result)
        if ttest_result[1]<0.05:
            print(f'*** Significant! head={hhh}, p={ttest_result[1]}, prop={prop}')
        else:
            print('- not significant...')
                
        table.append(['Proportion ' + prop if 'is_' in prop else prop, 
                     f"{means[0]:.4f}+-{stds[0]:.4f}", 
                     f"{means[1]:.4f}+-{stds[1]:.4f}", 
                     ttest_result[1],
                     hhh]
                    )
        df_by_prop.append(df_minimum)
    table_by_pdb.append(pd.concat(df_by_prop,axis=1).assign(head=hhh))
    



head 0 ******************************

  prop is_connecting_to_pep
	 mean+std of large = 0.02564102564102564+-0.15806189751202504
	 mean+std of small = 0.0853103218264868+-0.05767989280230504
Ttest_relResult(statistic=-2.0940523732318352, pvalue=0.04297911893950941)
*** Significant! head=0, p=0.04297911893950941, prop=is_connecting_to_pep

  prop is_connecting_to_cdr
	 mean+std of large = 0.6068376068376068+-0.426152166233091
	 mean+std of small = 0.4134740167962019+-0.10610517457341109
Ttest_relResult(statistic=2.6707454896397294, pvalue=0.011078143038863845)
*** Significant! head=0, p=0.011078143038863845, prop=is_connecting_to_cdr

  prop is_connecting_to_ownchain_tcr
	 mean+std of large = 0.7051282051282052+-0.4037952755903492
	 mean+std of small = 0.6407510013418598+-0.08882767424403212
Ttest_relResult(statistic=0.9409280815426917, pvalue=0.3526850937316177)
- not significant...

  prop is_connecting_to_ownchain_cdr
	 mean+std of large = 0.6068376068376068+-0.426152166233091
	 m


  prop distance_value
	 mean+std of large = 7.929343000346538+-4.017825244056091
	 mean+std of small = 8.3982443442939+-1.0153429868585664
Ttest_relResult(statistic=-0.5469558082029503, pvalue=0.5880890103914004)
- not significant...

  prop num_bonds
	 mean+std of large = 2.176470588235294+-1.818990767577901
	 mean+std of small = 2.074649172001457+-0.6641805025352525
Ttest_relResult(statistic=0.5193623675573908, pvalue=0.6069742478681885)
- not significant...


head 3 ******************************

  prop is_connecting_to_pep
	 mean+std of large = 0.08771929824561403+-0.24701978667243374
	 mean+std of small = 0.08176845533590238+-0.05724128309371131
Ttest_relResult(statistic=0.11523630600877662, pvalue=0.908880956151216)
- not significant...

  prop is_connecting_to_cdr
	 mean+std of large = 0.4342105263157895+-0.4318115547742615
	 mean+std of small = 0.42339366195553485+-0.10131698622163132
Ttest_relResult(statistic=0.18849861951755867, pvalue=0.8515158963382232)
- not significant.

In [39]:
df = pd.DataFrame(table, 
             columns=['property', 'Large Atten Mean. Mean(STD)', 'Small Atten. Mean(STD)', 'P Value', 'Head'])
df.loc[df['P Value'] < 0.10, ' '] = '*'
df.loc[df['P Value'] < 0.05, ' '] = '***'
df.loc[df['P Value'] >= 0.05, ' '] = ''
df[' '].fillna(' ')
df

,property,Large Atten Mean. Mean(STD),Small Atten. Mean(STD),P Value,Head,
0,Proportion is_connecting_to_pep,0.0256+-0.1581,0.0853+-0.0577,0.043,0,***
1,Proportion is_connecting_to_cdr,0.6068+-0.4262,0.4135+-0.1061,0.0111,0,***
2,Proportion is_connecting_to_ownchain_tcr,0.7051+-0.4038,0.6408+-0.0888,0.353,0,
3,Proportion is_connecting_to_ownchain_cdr,0.6068+-0.4262,0.3792+-0.0991,0.0027,0,***
4,Proportion is_connecting_to_opposite_chain_tcr,0.1111+-0.2833,0.1565+-0.0756,0.371,0,
5,Proportion is_connecting_to_opposite_chain_cdr,0.0000+-0.0000,0.0642+-0.0673,8.37e-07,0,***
6,Proportion is_connecting_to_tcr,0.7308+-0.3897,0.7193+-0.0903,0.864,0,
7,Proportion is_connecting_to_notCDR_tcr,0.1966+-0.3734,0.4479+-0.0738,0.000482,0,***
8,Proportion digit4_is_in_edge,0.8376+-0.3235,0.5904+-0.0387,5.1e-05,0,***
9,distance_value,9.1216+-3.1274,8.3528+-1.0097,0.119,0,


In [40]:
!ls *csv

aa_codes.csv                          one_cluster_sampled_10January2023.csv
files_20211029_022414.csv             stats_overleaf.csv
nohbond_largeattention.csv


In [41]:
pd.concat(table_by_pdb, axis=0).to_parquet('stats_overleaf.csv')